# Basics of LLM applications

## Basic Components

The most basic components of any LLM application are the following:

- **Input**

As with any other program, we desire for any input by the user to be converted to a useful output through a series of predefined steps.

- **Setup functions**

The input must undergo a series of transformations in order to be used by the API

- **System Prompt**

This is a natural language description of the task the LLM will assist with, as well as instructions and specifications on how this task is to be achieved.

- **API**

This is the step that actually generates the response(s), which will then be displayed by the app through a frontend environment

## Assembling the components

These basic components can be assembled in many different ways depending on the desired outcome. One example of a simple architecture would be the following:

![simple_design](_assets/simple_design.png)

This example takes the initial API response as input for more rounds of generation, an approach that can be used both for response refinement and for the addition of "memory" to the model. This is just one very simple permutation of possible architectures that can be designed to meet the needs of the application.

## Creating a simple Comand Line Interface (CLI) app

We will now use the previously defined architecture to create an app that will answer a question, and then stores this answer in a chat history or "memory" to be used as context for further queries. A CLI app is one in which the frontend is the command line. In our case, we are using a Jupyter Notebook, which renders command line outputs in a more interactive manner.

To start off, you will need the `openai` and `tiktoken` package. Make sure to first install them if you haven't yet, and then you can import them.



In [1]:
# !pip install openai tiktoken
from openai import OpenAI # the tiktoken package is imported by openai while functioning, so no need to explicitly import it

### Acquiring and setting up an OpenAI API key

To use the OpenAI API, you must have an account and generate a key. Enter the [OpenAI website](https://platform.openai.com/account/api-keys) and follow the steps.

After generating a key, you will have to add as an environment variable for the API to work. The easiest way to do this is to create a Python file called `Constants.py` that contains the key as a string, and then importing this string into the current kernel. After these steps, you can now set the api key for openai to function.

In [2]:
from Constants import OPENAI_API_KEY

client = OpenAI(api_key = OPENAI_API_KEY)

### Handle user input

Here, we take the user input and convert it into something usable for the API. In a CLI, we would use the built-in `input` function to get the user's input as a string, but for the sake of using a Jupyter Notebook, we will explicitly change the input string instead.

We will be using the Chat Completion endpoint of the API, which takes a list of messages, each represented by a dictionary with two entries. The `"role"` entry of the dictionary specifies the role of the message's sender, which can be `"system"`, `"user"`, and `"assistant"`; the `"content"` entry contains the body of the message. Therefore, we must create this list with the necessary information.

First we create a system prompt and add it in a dictionary to the list of messages

In [3]:
system_prompt = "You will answer the user's question to the best of your abilities"
messages = [{"role": "system", "content": system_prompt}]

Then we get the user input and add it to the list

In [4]:
user_input = "what is an elephant?"
messages.append({"role": "user", "content": user_input})

Now we have the prompt and the query in the necessary format to be used with the API. It is also necessary to specify the model to be used with the `model` argument. In this case we will use the `"gpt-3.5-turbo-1106"` model.

In [5]:
model = "gpt-3.5-turbo-1106"
response = client.chat.completions.create(model = model, messages = messages)
response

ChatCompletion(id='chatcmpl-8fxwYk12Z7vjVxvJUnQiTeiE9OE5b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='An elephant is a large mammal known for its long trunk, tusks, and large ears. There are two main species of elephants: the African elephant and the Asian elephant. They are highly intelligent and social animals, known for their complex communication, strong family bonds, and remarkable memory. Elephants are herbivores and can be found in various habitats, including savannas, forests, and grasslands.', role='assistant', function_call=None, tool_calls=None))], created=1705012562, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_cbe4fa03fe', usage=CompletionUsage(completion_tokens=83, prompt_tokens=29, total_tokens=112))

This returns a `ChatCompletion` class object with a lot of information on the response that has been returned. We can retrieve the response as a string by selecting a choice from the `choices` attribute of this class, which is a list of `Choice` class objects. Each of these objects has a `message` attribute which is a `ChatCompletionMessage` class object that contains the message and some information about it. The message string is stored in its `content` attribute.

In [6]:
response.choices[0].message.content

'An elephant is a large mammal known for its long trunk, tusks, and large ears. There are two main species of elephants: the African elephant and the Asian elephant. They are highly intelligent and social animals, known for their complex communication, strong family bonds, and remarkable memory. Elephants are herbivores and can be found in various habitats, including savannas, forests, and grasslands.'

We can now add this message to the list of messages, such that we can build a chat history for the model to use.

In [7]:
messages.append({"role": "assistant", "content": response.choices[0].message.content})

Now we can reference previous responses and the assistant will be able to understand the context

In [8]:
user_input = "do they ever attack humans?"
messages.append({"role": "user", "content": user_input})
response = client.chat.completions.create(model = model, messages = messages)
response.choices[0].message.content

"In some cases, elephants have been known to attack humans, but these incidents are typically a result of feeling threatened or provoked. Elephants are generally peaceful animals and will avoid confrontation if possible. However, conflicts can arise in situations where humans encroach on their natural habitat, or if an elephant feels their young are threatened. It's important to respect their space and behavior to avoid such confrontations. Conservation efforts and responsible wildlife management aim to protect both elephants and humans from these types of interactions."